In [1]:
import numpy as np
import random
from scipy.stats import norm
from sklearn.mixture import GaussianMixture

In [2]:
def drawFromADist(p):

    if np.sum(p) == 0 :
        p = 0.05 * np.ones((1,len(p)))

    p = p / (np.sum(p))
    c = np.cumsum(p)

    idx = np.where((np.random.uniform()- c)<0)
    ##print(idx)
    sample = np.min(idx)
    ##print(p.shape)
    out = np.zeros(len(p))
    ##print(out)
    out[sample] = 1

    return out


# PART 1

In [3]:
def tcm():
    N_WORLD_FEATURES = 5
    N_ITEMS = 10
    ENCODING_TIME = 500
    TEST_TIME = 20

    '''% we are going to model the world as a set of N continuous-valued features.
    % we will model observations of states of the world as samples from N
    % Gaussians with time-varying means and fixed variance. For simplicity,
    % assume that agents change nothing in the world.

    % first fix the presentation schedule; I am assuming its random'''

    #schedule = sorted(random.sample([i for i in range(1,ENCODING_TIME+1)],N_ITEMS))
    schedule = [2, 14, 25, 61, 153, 261, 269, 272, 462, 464] ## Fixed Schedule for which we get almost average success rate of 7

    schedule_load = ENCODING_TIME/np.median(np.diff(schedule))                ##% variable important for parts 2,3 of assignment
    encoding = np.zeros((N_ITEMS,N_WORLD_FEATURES+1))

##    world_m = np.array(random.choices([i for i in range(1,6)], k = N_WORLD_FEATURES))  ## Initial Feature means % can generate randomly for yourself
##    print(world_m)
    world_m = np.array([5, 3, 1, 5, 1]) ##Fixed initial world means for which we get nearly average success rate of 7
    
    world_var = 1
    delta = 0.05                      ## % what does this parameter affect? Amount of change in world features values per unit of time 
    beta_param = 0.001                ## % what does this parameter affect? Weight of new sampled world features values in new world state 
    m = 0

    world = np.array([0]*5)## Initial world state

    
    ##% simulating encoding

    for time in range(1,ENCODING_TIME+1):
        world_m = world_m + delta
        temp = world_m + world_var*np.random.randn(N_WORLD_FEATURES)
        prod = np.dot(temp,world)
        p = (1+(beta_param**2)*((prod**2)-1))**0.5 - beta_param*prod
        world = p*world + beta_param*temp
        ##% any item I want to encode in memory, I encode in association with the
        ##% state of the world at that time.
        if m<N_ITEMS :
            if(time==schedule[m]):
                encoding[m,:] = np.append(world,m)            ##% encode into the encoding vector
                m =  m + 1

    ##% simulating retrieval using SAM, but with a bijective image-item mapping


    out = [0]*TEST_TIME
    while(time<ENCODING_TIME+TEST_TIME):
        
    ##% the state of the world is the retrieval cue
        world_m = world_m + delta
        temp = world_m + world_var*np.random.randn(N_WORLD_FEATURES)
        prod = np.dot(temp,world)
        p = (1+(beta_param**2)*((prod**2)-1))**0.5 - beta_param*prod
        world = p*world + beta_param*temp
        soa = [0]*N_ITEMS
        for m in range(N_ITEMS):
        
            soa[m] = np.dot(encoding[m,:5], world)    ## % finding association strengths

        soa = soa/np.sum(soa)                                                                ## % normalize
        out[time-ENCODING_TIME] = np.where(drawFromADist(soa)==1)
       
        time = time + 1
    
    success= np.unique(out)     ##% success is number of unique retrievals

##    if(len(success)==7):
##        print(schedule)
   
    return schedule_load, len(success)


In [4]:
ch = input('1. Single Trial \n2. Multiple Trials \n')

if(ch == '1'):
    schedule_load, nunique = tcm()
    print('No of unique retrivals :', nunique)
    print('Scheduling load : ',schedule_load)

elif(ch == '2'):
     
     sch = []
     uniq= []
     for i in range(100):
         schedule_load, nunique = tcm()
         sch.append(schedule_load)
         uniq.append(nunique)
         print('No of unique retrivals for',i,'trial  :', nunique)
         
     print('\n')
     print('Schedule Load :',np.mean(sch))
     print('Mean Unique Retrivals :', np.mean(uniq))
         
else:
    print('Invalid Choice')

###% humans can retrieve about 7 items effectively from memory. get this model % to behave like humans'''

1. Single Trial 
2. Multiple Trials 
2
No of unique retrivals for 0 trial  : 7
No of unique retrivals for 1 trial  : 8
No of unique retrivals for 2 trial  : 7
No of unique retrivals for 3 trial  : 6
No of unique retrivals for 4 trial  : 8
No of unique retrivals for 5 trial  : 8
No of unique retrivals for 6 trial  : 7
No of unique retrivals for 7 trial  : 7
No of unique retrivals for 8 trial  : 8
No of unique retrivals for 9 trial  : 6
No of unique retrivals for 10 trial  : 8
No of unique retrivals for 11 trial  : 7
No of unique retrivals for 12 trial  : 9
No of unique retrivals for 13 trial  : 6
No of unique retrivals for 14 trial  : 7
No of unique retrivals for 15 trial  : 8
No of unique retrivals for 16 trial  : 7
No of unique retrivals for 17 trial  : 8
No of unique retrivals for 18 trial  : 9
No of unique retrivals for 19 trial  : 8
No of unique retrivals for 20 trial  : 9
No of unique retrivals for 21 trial  : 8
No of unique retrivals for 22 trial  : 6
No of unique retrivals for 2

### In above experimentation we get Average Success Retrival rate of approximalety 7.25 to 7.7 and Schedule load of 41.667

# PART 2 (A)

In [5]:
def compdelta():
    x = [norm(0.2, 0.5), norm(5, 1)]
    c = np.random.choice([0, 1], p=[0.6, 0.4])
    return x[c].rvs()

In [6]:
def tcm():
    N_WORLD_FEATURES = 5
    N_ITEMS = 10
    ENCODING_TIME = 500
    TEST_TIME = 20

    '''% we are going to model the world as a set of N continuous-valued features.
    % we will model observations of states of the world as samples from N
    % Gaussians with time-varying means and fixed variance. For simplicity,
    % assume that agents change nothing in the world.

    % first fix the presentation schedule; I am assuming its random'''

 
    schedule = [2, 14, 25, 61, 153, 261, 269, 272, 462, 464] ## Fixed Schedule for which we get almost average success rate of 7
    schedule_load = ENCODING_TIME/np.median(np.diff(schedule))                ##% variable important for parts 2,3 of assignment
    encoding = np.zeros((N_ITEMS,N_WORLD_FEATURES+1))

    world_m = np.array([5, 3, 1, 5, 1]) ##Fixed initial world means for which we get nearly average success rate of 7
    
    world_var = 1
    beta_param = 0.001                ## % what does this parameter affect? Weight of new sampled world features values in new world state 
    m = 0

    world = np.array([0]*5)## Initial world state

    
    ##% simulating encoding

    for time in range(1,ENCODING_TIME+1):
        delta = compdelta()
        world_m = world_m + delta
        temp = world_m + world_var*np.random.randn(N_WORLD_FEATURES)
        prod = np.dot(temp,world)
        p = (1+(beta_param**2)*((prod**2)-1))**0.5 - beta_param*prod
        world = p*world + beta_param*temp
        ##% any item I want to encode in memory, I encode in association with the
        ##% state of the world at that time.
        if m<N_ITEMS :
            if(time==schedule[m]):
                encoding[m,:] = np.append(world,m)            ##% encode into the encoding vector
                m =  m + 1

    ##% simulating retrieval using SAM, but with a bijective image-item mapping


    out = [0]*TEST_TIME
    while(time<ENCODING_TIME+TEST_TIME):
        
    ##% the state of the world is the retrieval cue
        delta = compdelta()
        world_m = world_m + delta
        temp = world_m + world_var*np.random.randn(N_WORLD_FEATURES)
        prod = np.dot(temp,world)
        p = (1+(beta_param**2)*((prod**2)-1))**0.5 - beta_param*prod
        world = p*world + beta_param*temp
        soa = [0]*N_ITEMS
        for m in range(N_ITEMS):
        
            soa[m] = np.dot(encoding[m,:5], world)    ## % finding association strengths

        soa = soa/np.sum(soa)                                                                ## % normalize
        out[time-ENCODING_TIME] = np.where(drawFromADist(soa)==1)
       
        time = time + 1
    
    success= np.unique(out)     ##% success is number of unique retrievals


    return schedule_load, len(success)




In [7]:
ch = input('1. Single Trial \n2. Multiple Trials \n')

if(ch == '1'):
    schedule_load, nunique = tcm()
    print('No of unique retrivals :', nunique)
    print('Scheduling load : ',schedule_load)

elif(ch == '2'):

     uniq= []
     for i in range(100):
         schedule_load, nunique = tcm()
         sch = schedule_load
         uniq.append(nunique)
     
     print('\n')    
     print('Schedule Load :',sch)
     print('Mean Unique Retrivals :', np.mean(uniq))
         
else:
    print('Invalid Choice')

##% humans can retrieve about 7 items effectively from memory. get this model
##% to behave like humans

1. Single Trial 
2. Multiple Trials 
2


Schedule Load : 41.666666666666664
Mean Unique Retrivals : 7.77


# PART 2 (B) 

### I have assumed independent gaussian models for the mixture from which delta is being sampled

In [8]:
def tcm(schedule):
    N_WORLD_FEATURES = 5
    N_ITEMS = 10
    ENCODING_TIME = 500
    TEST_TIME = 20

    '''% we are going to model the world as a set of N continuous-valued features.
    % we will model observations of states of the world as samples from N
    % Gaussians with time-varying means and fixed variance. For simplicity,
    % assume that agents change nothing in the world.

    % first fix the presentation schedule; I am assuming its random'''


    schedule_load = ENCODING_TIME/np.median(np.diff(schedule))                ##% variable important for parts 2,3 of assignment
    encoding = np.zeros((N_ITEMS,N_WORLD_FEATURES+1))

    world_m = np.array([5, 3, 1, 5, 1]) ##Fixed initial world means for which we get nearly average success rate of 7
    
    world_var = 1
    beta_param = 0.001                ## % what does this parameter affect? Weight of new sampled world features values in new world state 
    m = 0

    world = np.array([0]*5)## Initial world state

    
    ##% simulating encoding

    for time in range(1,ENCODING_TIME+1):
        delta = compdelta()
        world_m = world_m + delta
        temp = world_m + world_var*np.random.randn(N_WORLD_FEATURES)
        prod = np.dot(temp,world)
        p = (1+(beta_param**2)*((prod**2)-1))**0.5 - beta_param*prod
        world = p*world + beta_param*temp
        ##% any item I want to encode in memory, I encode in association with the
        ##% state of the world at that time.
        if m<N_ITEMS :
            if(time==schedule[m]):
                encoding[m,:] = np.append(world,m)            ##% encode into the encoding vector
                m =  m + 1

    ##% simulating retrieval using SAM, but with a bijective image-item mapping


    out = [0]*TEST_TIME
    while(time<ENCODING_TIME+TEST_TIME):
        
    ##% the state of the world is the retrieval cue
        delta = compdelta()
        world_m = world_m + delta
        temp = world_m + world_var*np.random.randn(N_WORLD_FEATURES)
        prod = np.dot(temp,world)
        p = (1+(beta_param**2)*((prod**2)-1))**0.5 - beta_param*prod
        world = p*world + beta_param*temp
        soa = [0]*N_ITEMS
        for m in range(N_ITEMS):
        
            soa[m] = np.dot(encoding[m,:5], world)    ## % finding association strengths

        soa = soa/np.sum(soa)                                                                ## % normalize
        out[time-ENCODING_TIME] = np.where(drawFromADist(soa)==1)
       
        time = time + 1
    
    success= np.unique(out)     ##% success is number of unique retrievals


    return schedule_load, len(success)

### Schedule Load is inversely propptional to median of difference of consecutive items's encoding time, i.e. if the median of difference increases, schedule load will decrease. In our case there are 10 items and therefore 9 differences. To have maximum median of difference we will keep  4 differences to be minimum and  5 differences to be maximum keeping in mind that our average success rate is atleast 7.

In [9]:
def schedulelist():
    schlist = []
    for small in range(1,11,2):
        big = (499-4*small)//5
        schlist.append([1,big+1,2*big+1,3*big+1,4*big+1,5*big+1,5*big+1+small,5*big+1+small*2,5*big+1+small*3,5*big+1+small*4])
        schlist.append([1,small+1,2*small+1,3*small+1,4*small+1,4*small+1+big,4*small+1+big*2,4*small+1+big*3,4*small+1+big*4,4*small+1+big*5])

    return schlist

In [10]:
schlist = schedulelist()
schload = []
sch = []
succ = []
ch = input('1. Single Trial \n2. Multiple Trials \n')
print()
if(ch == '1'):
    for schedule in schlist:
        schedule_load, nunique = tcm(schedule)
        print('Schedule: ',schedule)
        print('No of unique retrivals :', nunique)
        print('Scheduling load : ',schedule_load)
        print()
        if nunique >= 7:
            schload.append(schedule_load)
            sch.append(schedule)
            succ.append(nunique)
        
    print('\n\n\nSchedule with lowest schedule load and average success retrival rate atleast  7')
    minload = np.argmin(schload)
    print('Schedule :',sch[minload])
    print('Schedule Load :',schload[minload])
    print('No of unique retrivals :', succ[minload])
        

elif(ch == '2'):
     
    for schedule in schlist: 
        tempsch = 0
        uniq= []
        for i in range(100):
            schedule_load, nunique = tcm(schedule)
            tempsch= schedule_load
            uniq.append(nunique)
         
        print('Schedule:',schedule)
        print('Schedule Load :',tempsch)
        print('Mean Unique Retrivals :', np.mean(uniq))
        print()
        if np.mean(uniq) >= 7:
            schload.append(schedule_load)
            sch.append(schedule)
            succ.append(np.mean(uniq))
            
    print('\n\n\nSchedule with lowest schedule load and average success retrival rate atleast  7')
    minload = np.argmin(schload)
    print('Schedule :',sch[minload])
    print('Schedule Load :',schload[minload])
    print('Mean No of unique retrivals :', succ[minload])
         
else:
    print('Invalid Choice')


1. Single Trial 
2. Multiple Trials 
2

Schedule: [1, 100, 199, 298, 397, 496, 497, 498, 499, 500]
Schedule Load : 5.05050505050505
Mean Unique Retrivals : 7.95

Schedule: [1, 2, 3, 4, 5, 104, 203, 302, 401, 500]
Schedule Load : 5.05050505050505
Mean Unique Retrivals : 5.38

Schedule: [1, 98, 195, 292, 389, 486, 489, 492, 495, 498]
Schedule Load : 5.154639175257732
Mean Unique Retrivals : 7.92

Schedule: [1, 4, 7, 10, 13, 110, 207, 304, 401, 498]
Schedule Load : 5.154639175257732
Mean Unique Retrivals : 6.24

Schedule: [1, 96, 191, 286, 381, 476, 481, 486, 491, 496]
Schedule Load : 5.2631578947368425
Mean Unique Retrivals : 7.98

Schedule: [1, 6, 11, 16, 21, 116, 211, 306, 401, 496]
Schedule Load : 5.2631578947368425
Mean Unique Retrivals : 7.01

Schedule: [1, 95, 189, 283, 377, 471, 478, 485, 492, 499]
Schedule Load : 5.319148936170213
Mean Unique Retrivals : 8.16

Schedule: [1, 8, 15, 22, 29, 123, 217, 311, 405, 499]
Schedule Load : 5.319148936170213
Mean Unique Retrivals : 7.3

Sche

### It is observed after repetitive experimentation with above code that a schedule with 

#### (i) 5 big differences (in range 91 to 95) at beginning of schedule and 4 small differences (in range 10 to 5) at the end generally gives  an average success retrival rate of greater than equal to 7

#### [1, 96, 191, 286, 381, 476, 481, 486, 491, 496],[1, 95, 189, 283, 377, 471, 478, 485, 492, 499],[1, 92, 183, 274, 365, 456, 466, 476, 486,496]

#### (ii) 4 small differences (in range 7 to 10) at beginning of schedule and 5 big differences (in range 95 to 91) at end generally gives an averafe success retrival rate of gretaer than equal to 7

#### [1, 8, 15, 22, 29, 123, 217, 311, 405, 499] , [1, 9, 17, 25, 33, 126, 219, 312, 405, 498] , [1, 11, 21, 31, 41, 132, 223, 314, 405, 496]

# PART 3 (A)

### I have assumed independent gaussian models for the mixture from which delta is being sampled

In [11]:
def compdelta(): ##Samples original GMM 
    x = [norm(0.2, 0.5), norm(5, 1)]
    c = np.random.choice([0, 1], p=[0.6, 0.4])
    return x[c].rvs()

In [12]:
def compdelta2(m1,v1,w1,m2,v2,w2): ##Samples Estimated GMM using EM Algorithm
    x = [norm(m1, v1), norm(m2, v2)]
    c = np.random.choice([0, 1], p=[w1, w2])
    return x[c].rvs()

In [13]:
def tcm():
    N_WORLD_FEATURES = 5
    N_ITEMS = 10
    ENCODING_TIME = 500
    TEST_TIME = 20

    '''% we are going to model the world as a set of N continuous-valued features.
    % we will model observations of states of the world as samples from N
    % Gaussians with time-varying means and fixed variance. For simplicity,
    % assume that agents change nothing in the world.

    % first fix the presentation schedule; I am assuming its random'''

 
    schedule = [2, 14, 25, 61, 153, 261, 269, 272, 462, 464] ## Fixed Schedule for which we get almost average success rate of 7
    schedule_load = ENCODING_TIME/np.median(np.diff(schedule))                ##% variable important for parts 2,3 of assignment
    encoding = np.zeros((N_ITEMS,N_WORLD_FEATURES+1))

    world_m = np.array([5, 3, 1, 5, 1]) ##Fixed initial world means for which we get nearly average success rate of 7
    
    world_var = 1
    beta_param = 0.001                ## % what does this parameter affect? Weight of new sampled world features values in new world state 
    m = 0

    world = np.array([0]*5)## Initial world state

    
    ##% simulating encoding
    ogdelta = []
    for time in range(1,ENCODING_TIME+1):
        delta = compdelta()
        ogdelta.append(delta)
        world_m = world_m + delta
        temp = world_m + world_var*np.random.randn(N_WORLD_FEATURES)
        prod = np.dot(temp,world)
        p = (1+(beta_param**2)*((prod**2)-1))**0.5 - beta_param*prod
        world = p*world + beta_param*temp
        ##% any item I want to encode in memory, I encode in association with the
        ##% state of the world at that time.
        if m<N_ITEMS :
            if(time==schedule[m]):
                encoding[m,:] = np.append(world,m)            ##% encode into the encoding vector
                m =  m + 1
    ogdelta = np.array([[o] for o in ogdelta])       
    gmm = GaussianMixture(n_components=2, covariance_type='spherical',tol=1e-10,weights_init = [0.5,0.5], means_init = [[0],[3]],precisions_init = [0.25,0.75])
    gmm.fit(ogdelta)
    mu = gmm.means_
    var = gmm.covariances_
    w = gmm.weights_
 
    ##% simulating retrieval using SAM, but with a bijective image-item mapping

    out = [0]*TEST_TIME
    while(time<ENCODING_TIME+TEST_TIME):
        delta = compdelta2(mu[0],var[0],w[0],mu[1],var[1],w[1])
        world_m = world_m + delta
        temp = world_m + world_var*np.random.randn(N_WORLD_FEATURES)
        prod = np.dot(temp,world)
        p = (1+(beta_param**2)*((prod**2)-1))**0.5 - beta_param*prod
        world = p*world + beta_param*temp
        soa = [0]*N_ITEMS
        for m in range(N_ITEMS):
        
            soa[m] = np.dot(encoding[m,:5], world)    ## % finding association strengths

        soa = soa/np.sum(soa)                                                                ## % normalize
        out[time-ENCODING_TIME] = np.where(drawFromADist(soa)==1)
       
        time = time + 1
    
    success= np.unique(out)     ##% success is number of unique retrievals


    return schedule_load, len(success)




In [14]:
ch = input('1. Single Trial \n2. Multiple Trials \n')

if(ch == '1'):
    schedule_load, nunique = tcm()
    print('No of unique retrivals :', nunique)
    print('Scheduling load : ',schedule_load)

elif(ch == '2'):

     uniq= []
     for i in range(100):
         schedule_load, nunique = tcm()
         sch = schedule_load
         uniq.append(nunique)
     
     print('\n')    
     print('Schedule Load :',sch)
     print('Mean Unique Retrivals :', np.mean(uniq))
         
else:
    print('Invalid Choice')

##% humans can retrieve about 7 items effectively from memory. get this model
##% to behave like humans

1. Single Trial 
2. Multiple Trials 
2


Schedule Load : 41.666666666666664
Mean Unique Retrivals : 7.71


# PART 3 (B)

In [15]:
def schedulelist():
    schlist = []
    for small in range(1,11,2):
        big = (499-4*small)//5
        schlist.append([1,big+1,2*big+1,3*big+1,4*big+1,5*big+1,5*big+1+small,5*big+1+small*2,5*big+1+small*3,5*big+1+small*4])
        schlist.append([1,small+1,2*small+1,3*small+1,4*small+1,4*small+1+big,4*small+1+big*2,4*small+1+big*3,4*small+1+big*4,4*small+1+big*5])

    return schlist

In [16]:
def tcm(schedule):
    N_WORLD_FEATURES = 5
    N_ITEMS = 10
    ENCODING_TIME = 500
    TEST_TIME = 20

    '''% we are going to model the world as a set of N continuous-valued features.
    % we will model observations of states of the world as samples from N
    % Gaussians with time-varying means and fixed variance. For simplicity,
    % assume that agents change nothing in the world.

    % first fix the presentation schedule; I am assuming its random'''

 
    
    schedule_load = ENCODING_TIME/np.median(np.diff(schedule))                ##% variable important for parts 2,3 of assignment
    encoding = np.zeros((N_ITEMS,N_WORLD_FEATURES+1))

    world_m = np.array([5, 3, 1, 5, 1]) ##Fixed initial world means for which we get nearly average success rate of 7
    
    world_var = 1
    beta_param = 0.001                ## % what does this parameter affect? Weight of new sampled world features values in new world state 
    m = 0

    world = np.array([0]*5)## Initial world state

    
    ##% simulating encoding
    ogdelta = []
    for time in range(1,ENCODING_TIME+1):
        delta = compdelta()
        ogdelta.append(delta)
        world_m = world_m + delta
        temp = world_m + world_var*np.random.randn(N_WORLD_FEATURES)
        prod = np.dot(temp,world)
        p = (1+(beta_param**2)*((prod**2)-1))**0.5 - beta_param*prod
        world = p*world + beta_param*temp
        ##% any item I want to encode in memory, I encode in association with the
        ##% state of the world at that time.
        if m<N_ITEMS :
            if(time==schedule[m]):
                encoding[m,:] = np.append(world,m)            ##% encode into the encoding vector
                m =  m + 1
    ogdelta = np.array([[o] for o in ogdelta])       
    gmm = GaussianMixture(n_components=2, covariance_type='spherical',tol=1e-10,weights_init = [0.5,0.5], means_init = [[0],[3]],precisions_init = [0.25,0.75])
    gmm.fit(ogdelta)
    mu = gmm.means_
    var = gmm.covariances_
    w = gmm.weights_
 
    ##% simulating retrieval using SAM, but with a bijective image-item mapping

    out = [0]*TEST_TIME
    while(time<ENCODING_TIME+TEST_TIME):
        delta = compdelta2(mu[0],var[0],w[0],mu[1],var[1],w[1])
        world_m = world_m + delta
        temp = world_m + world_var*np.random.randn(N_WORLD_FEATURES)
        prod = np.dot(temp,world)
        p = (1+(beta_param**2)*((prod**2)-1))**0.5 - beta_param*prod
        world = p*world + beta_param*temp
        soa = [0]*N_ITEMS
        for m in range(N_ITEMS):
        
            soa[m] = np.dot(encoding[m,:5], world)    ## % finding association strengths

        soa = soa/np.sum(soa)                                                                ## % normalize
        out[time-ENCODING_TIME] = np.where(drawFromADist(soa)==1)
       
        time = time + 1
    
    success= np.unique(out)     ##% success is number of unique retrievals


    return schedule_load, len(success)




In [ ]:
schlist = schedulelist()
schload = []
sch = []
succ = []
ch = input('1. Single Trial \n2. Multiple Trials \n')
print()
if(ch == '1'):
    for schedule in schlist:
        schedule_load, nunique = tcm(schedule)
        print('Schedule: ',schedule)
        print('No of unique retrivals :', nunique)
        print('Scheduling load : ',schedule_load)
        print()
        if nunique >= 7:
            schload.append(schedule_load)
            sch.append(schedule)
            succ.append(nunique)
        
    print('\n\n\nSchedule with lowest schedule load and average success retrival rate atleast  7')
    minload = np.argmin(schload)
    print('Schedule :',sch[minload])
    print('Schedule Load :',schload[minload])
    print('No of unique retrivals :', succ[minload])
        

elif(ch == '2'):
     
    for schedule in schlist: 
        tempsch = 0
        uniq= []
        for i in range(100):
            schedule_load, nunique = tcm(schedule)
            tempsch= schedule_load
            uniq.append(nunique)
         
        print('Schedule:',schedule)
        print('Schedule Load :',tempsch)
        print('Mean Unique Retrivals :', np.mean(uniq))
        print()
        if np.mean(uniq) >= 7:
            schload.append(schedule_load)
            sch.append(schedule)
            succ.append(np.mean(uniq))
            
    print('\n\n\nSchedule with lowest schedule load and average success retrival rate atleast  7')
    minload = np.argmin(schload)
    print('Schedule :',sch[minload])
    print('Schedule Load :',schload[minload])
    print('Mean No of unique retrivals :', succ[minload])
         
else:
    print('Invalid Choice')

1. Single Trial 
2. Multiple Trials 
2



### It is observed after repetitive experimentation with above code that a schedule with 

#### (i) 5 big differences (in range 91 to 95) at beginning of schedule and 4 small differences (in range 10 to 5) at the end generally gives  an average success retrival rate of greater than equal to 7

#### [1, 95, 189, 283, 377, 471, 478, 485, 492, 499] gives average success rate above 7 in most iterations

#### (ii) 4 small differences (in range 7 to 10) at beginning of schedule and 5 big differences (in range 95 to 91) at end generally gives an averafe success retrival rate of gretaer than equal to 7

#### [1, 9, 17, 25, 33, 126, 219, 312, 405, 498] gives average success rate above 7 in most iterations